<a href="https://colab.research.google.com/github/sandeeptuluri/fresh1/blob/main/all_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries.....

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Reading the CSV....

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/sandeeptuluri/fresh1/main/Data%20extract%20for%20Sandeep.csv')
df.head()

,id,summary,topicID,topicName
0,26696,Expo 2020 Dubai is finally open after an 8-mon...,13,Technology
1,26694,The EU has announced a humanitarian aid packag...,3,Politics
2,26693,Dutch royal weddings need the approval of parl...,3,Politics
3,26692,Governor Sheinbaum has confirmed that a statue...,10,North America
4,26690,US officials are investigating possible cases ...,3,Politics


In [3]:
df.isnull().sum()

id           0
summary      0
topicID      0
topicName    0
dtype: int64

In [4]:
df.topicName.nunique()

29

In [5]:
df.describe()


,id,topicID
count,12802.000000,12802.000000
mean,18746.503359,9.722231
std,4643.310327,6.670339
min,10858.000000,2.000000
25%,14614.250000,5.000000
50%,18707.500000,7.000000
75%,22829.750000,14.000000
max,26696.000000,33.000000


Data is very clean....and ready to use

In [6]:
df['summary'] = df['summary'].apply(lambda summary: (summary).lower())

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = df['summary']
y = df['topicID']

Splitting the data for efficient use....

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [10]:
from tensorflow.keras.utils import to_categorical


In [11]:
ytr = to_categorical(y_train)


In [12]:
ytr.shape

(8961, 34)

In [13]:
yte = to_categorical(y_test)

In [14]:
yte.shape

(3841, 34)

In [15]:
import tensorflow as tf


In [16]:
import tensorflow_hub as hub

Importing the pretrained embedded layer from google....for better training the model...

In [17]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
#hub_layer(train)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

Preparing the model....

In [19]:
model = Sequential()
model.add(hub_layer)
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(34, activation='softmax'))

In [20]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Fitting the model....

In [ ]:
model.fit(X_train,ytr,epochs=10,validation_data=(X_test,yte),verbose=1)

Epoch 1/10
281/281 [==============================] - 155s 547ms/step - loss: 2.4628 - accuracy: 0.3177 - val_loss: 2.0479 - val_accuracy: 0.4124
Epoch 2/10
281/281 [==============================] - 153s 546ms/step - loss: 1.8249 - accuracy: 0.4766 - val_loss: 1.9663 - val_accuracy: 0.4225
Epoch 3/10
281/281 [==============================] - 155s 551ms/step - loss: 1.3860 - accuracy: 0.5852 - val_loss: 2.1984 - val_accuracy: 0.4087
Epoch 4/10
281/281 [==============================] - 156s 555ms/step - loss: 1.0110 - accuracy: 0.6843 - val_loss: 2.4364 - val_accuracy: 0.3861
Epoch 5/10
281/281 [==============================] - 154s 549ms/step - loss: 0.7334 - accuracy: 0.7729 - val_loss: 3.2866 - val_accuracy: 0.3640
Epoch 6/10
281/281 [==============================] - 153s 545ms/step - loss: 0.5277 - accuracy: 0.8332 - val_loss: 4.0252 - val_accuracy: 0.3851
Epoch 7/10
281/281 [==============================] - 154s 548ms/step - loss: 0.3740 - accuracy: 0.8871 - val_loss: 4.2170 -

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics[['loss','val_loss']].plot()

In [ ]:
metrics[['accuracy','val_accuracy']].plot()

In [ ]:
model.metrics_names

evaluating the model.....

In [ ]:
model.evaluate(X_test,yte,verbose=0)

Predicting the model....

In [ ]:
predictions = model.predict(X_test)

In [ ]:
a = X_test[:1]

In [ ]:
pd.set_option("display.max_colwidth", -1)

In [52]:
a

5621    yeimo, a papuan activist was arrested and charged with treason for a 2019 statement, made during anti-racism protests that ensued riots.\nhrw says indonesia must drop politically motivated charges and release the activist.\nit urged president to direct those involved in counterinsurgency operations in papua to act in accordance with international law.
Name: summary, dtype: object

In [ ]:
h = model.predict(a)

In [ ]:
np.argmax(h)

Our top most prediction '3',  as politics... and it is True.

In [ ]:
np.argsort(h)

Here we have to consider the last three predictions.....as our main predictions....

Here the last three predictions(3,9,14) are the top most predictions.... i.e,(politics, travel, Politics).... and it is perfectly predicted

Here, the model predicted the accurate result.... and the accuracy is also quiet good....

In [ ]:
!mkdir -p saved_model
model.save('text_classification')

In [ ]:
ls

In [ ]:
ls saved_model/

In [ ]:
ls saved_model/text_classification

Follow from here......

Loading the model which has been already shared in the google colab.....

In [ ]:
import tensorflow as tf

In [ ]:
main = tf.keras.models.load_model('text_classification/')

Testing the model by picking the sample summary from the test dataset.....

Here we are picking the 13 summary from the test dataset.....

In [ ]:
k = X_test[12:13]

In [ ]:
k

In [ ]:
u = main.predict(k)

In [ ]:
np.argmax(u)

here our top most prediction is '14', and the it is 'travel'.

In [ ]:
np.argsort(u)

Top 3 predictions (14,2,7) i.e, (Travel, sports, Globalhealth)

code for keywords, sentiment and summary

In [53]:
pip install transformers

     |████████████████████████████████| 2.9 MB 7.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
     |████████████████████████████████| 895 kB 36.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [54]:
pip install newspaper3k

     |████████████████████████████████| 211 kB 8.8 MB/s 
     |████████████████████████████████| 87 kB 7.7 MB/s 
     |████████████████████████████████| 7.4 MB 37.9 MB/s 
     |████████████████████████████████| 81 kB 9.9 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=2d045236af95b51486ca89405a59ba4f5d7e5da2ec5564e9c73b58fc50a1541b
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3356 sha256=e97dc7b7cd43a188701cba50d34f88f0c22116943ad7dea59a2b7eaa7da5015e
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398405 sha256=72e4a254e43a78229ed6f9d0d09b753ebddd26343ceee39f7d14e3e8ab0c60da
  Stored in directory: /root/.cache/pip/wheels/4c/91/46/3c208287b726df325a597957432

In [72]:
pip install keywords

  Created wheel for keywords: filename=keywords-0.0.1-py3-none-any.whl size=3208 sha256=59ca43fbf43581d7aaa8db1cda8284dc110a79dd260b701b323c844ebddc2a93
  Stored in directory: /root/.cache/pip/wheels/ee/3e/71/ae67c09e02a0f1f221ebc49d1e3c8c255fd738d1edd15d76d7
Successfully built keywords


In [73]:
pip install textblob

In [55]:
import nltk
import string
import re
from newspaper import Article

In [56]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [58]:
url = 'https://www.thehindu.com/news/national/other-states/amit-shah-arrives-in-srinagar-for-3-day-visit-to-jammu-and-kashmir/article37136061.ece'

In [59]:
article = Article(url)
article.download()
article.parse()
article.nlp()

In [60]:
text = article.text

In [61]:
def clean_text(text):
  text = re.sub(r'[^a-zA-Z]', ' ', text)
  text = text.lower()
  text = text.split()
  text = [word for word in text if not word in set(nltk.corpus.stopwords.words('english'))]
  text = ' '.join(text)
  return text

In [63]:
#print(text)

In [74]:
from keybert import KeyBERT
from textblob import TextBlob
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration


model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

kb = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = kb.extract_keywords(text, stop_words='english')

analysis = TextBlob(text)
a = analysis.polarity

def type():
  if (a>0.2):
    return("Positive")
  elif (a<0):
    return("Negative")
  else:
    return("Neutral")

c = type()

preprocess_text = text.strip().replace("\n","")
t5_prepared_Text = "summarize: "+preprocess_text

inputs = tokenizer.encode("summarize: " + t5_prepared_Text, return_tensors='pt', max_length=512, truncation=True).to(device)

# summmarize
summary_ids = model.generate(inputs,num_beams=2,min_length=80,max_length=150,length_penalty=5.)
summary = tokenizer.decode(summary_ids[0],skip_special_tokens=True)

DEBUG:filelock:Attempting to acquire lock 140191149540304 on /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985.lock
DEBUG:filelock:Lock 140191149540304 acquired on /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985.lock


Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140191149540304 on /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985.lock
DEBUG:filelock:Lock 140191149540304 released on /root/.cache/huggingface/transformers/fe501e8fd6425b8ec93df37767fcce78ce626e34cc5edc859c662350cf712e41.406701565c0afd9899544c1cb8b93185a76f00b31e5ce7f6e18bbaef02241985.lock
DEBUG:filelock:Attempting to acquire lock 140187050048912 on /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885.lock
DEBUG:filelock:Lock 140187050048912 acquired on /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885.lock


Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140187050048912 on /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885.lock
DEBUG:filelock:Lock 140187050048912 released on /root/.cache/huggingface/transformers/fee5a3a0ae379232608b6eed45d2d7a0d2966b9683728838412caccc41b4b0ed.ddacdc89ec88482db20c676f0861a336f3d0409f94748c209847b49529d73885.lock
DEBUG:filelock:Attempting to acquire lock 140187048854864 on /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock
DEBUG:filelock:Lock 140187048854864 acquired on /root/.cache/huggingface/transformers/65fc04e21f45f61430aea0c4fedffac16a4d20d78b8e6601d8d996ebefefecd2.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock


...

DEBUG:filelock:Attempting to release lock 140187050014992 on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/tokenizer_config.json.lock
DEBUG:filelock:Lock 140187050014992 released on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/tokenizer_config.json.lock
DEBUG:filelock:Attempting to acquire lock 140187047540752 on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/vocab.txt.lock
DEBUG:filelock:Lock 140187047540752 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/vocab.txt.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140187047540752 on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/vocab.txt.lock
DEBUG:filelock:Lock 140187047540752 released on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/vocab.txt.lock
DEBUG:filelock:Attempting to acquire lock 140187049315920 on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/1_Pooling/config.json.lock
DEBUG:filelock:Lock 140187049315920 acquired on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/1_Pooling/config.json.lock


Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

DEBUG:filelock:Attempting to release lock 140187049315920 on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/1_Pooling/config.json.lock
DEBUG:filelock:Lock 140187049315920 released on /root/.cache/torch/sentence_transformers/sentence-transformers_distilbert-base-nli-mean-tokens/1_Pooling/config.json.lock


In [78]:
l = ["ProPublica says it has seen the tax returns of some of the world's richest people, including Jeff Bezos, Elon Musk and Warren Buffett.The website alleges Amazon's Mr Bezos paid no tax in 2007 and 2011, while Tesla's Mr Musk paid nothing in 2018.A White House spokeswoman called the leak illegal, and the FBI and tax authorities are investigating."]

In [79]:
z = main.predict(l)

In [80]:
np.argsort(z)

array([[23, 22,  1,  0, 17, 27,  2, 29, 15, 31, 32, 16,  4, 25, 33, 28,
        13,  7, 30, 24, 21,  5, 26, 19, 18, 14, 20,  6, 11, 10,  8, 12,
         3,  9]])

the top most predictions are (8,9,3) i.e, europe, asia, poitics........

you can check the results down below in the groupby table.....

In [ ]:
g = df.groupby(['topicID','topicName'])

In [ ]:
g.first()

,,id,summary
topicID,topicName,,
2,Sports,26632,"germany is set to play at the world cup 2022 in qatar after scaling a 4-0 victory over north macedonia.\nthey are the first side to qualify for next year’s tournament other than the hosts, who qualified automatically.\nit leaves them safely through from uefa qualifying group j with an eight-point lead over romania with two games to play."
3,Politics,26694,"the eu has announced a humanitarian aid package worth €1 billion to deal with the aftermath of the taliban takeover\n€300 million had been already agreed to support the country's civilian population, protect human rights and advance the coronavirus vaccination campaign\nthe european commission will now add an extra €250 million for similar purposes"
4,Start-ups,26612,"lego has had to “address gender bias and harmful stereotypes”, and since has promised to remove gender bias.\nlego is removing gender labels from its products and encouraging boys and girls to play with toys that may have traditionally been seen as not for them.\nit is also conducting research into how children perceive gender roles and stereotypes."
5,Business,26662,"adani ports, india’s largest port operator, said it will not handle cargo from afghanistan, pakistan and iran from next month.\nthe decision follows the seizure of three tonnes of heroin from two containers at the mundra port off india’s western coast in gujarat state last month.\nmost of the world’s opium and heroin comes from afghanistan."
6,Habitat,26676,"the greek island of crete has been hit a 6.4-magnitude earthquake, says the us geological survey, with the epicentre near the eastern tip.\nsome buildings were damaged, and thousands of people were seen leaving their homes in panic, with no reports of casualties so far.\ntremors were felt across several of the dodecanese islands."
7,Global health,26671,"the uk's failure to do more to stop covid spreading early in the pandemic was one of the country's worst public health failures, a report by mps says.\nthe government's approach was to try to manage the situation and achieve herd immunity by infection, the report says.\nthe report highlighted ""unacceptably high"" death rates in ethnic minority groups."
8,Europe,26678,"cemetery outside berlin said it made a “terrible mistake” by allowing the ashes of neo-nazi henry hafenmayer, to be buried in the gravesite of a jewish-born musicologist.\nhafenmayer had gained status in german right-wing extremist circles.\nhe had also been sentenced to prison over antisemitic letters that described the holocaust as a “lie”."
9,Asia,26688,leaders from g20 nations have stressed the importance of supporting afghanistan without recognizing the taliban.\ng20 leaders agreed that funding targets in humanitarian assistance should also focus on programs in favour of women and girls in afghanistan.\nthe taliban have ruled out cooperating with the us and other countries to fight terrorism.
10,North America,26692,"governor sheinbaum has confirmed that a statue of an indigenous woman will replace the christopher columbus monument, \nthe statue was removed last year after indigenous rights activists threatened to tear it down.\ncolumbus is seen by many as a symbol of oppression and colonialism as his arrival in america opened the door to the spanish conquest."


In [75]:
print(keywords)

[('terrorists', 0.1695), ('terrorism', 0.1179), ('saturday', 0.1042), ('october', 0.048), ('police', 0.0448)]


In [76]:
print(c)

Neutral


In [77]:
print(summary)

the visit comes in the backdrop of 11 civilian killings in the valley in October alone. five of those killed were labourers from Bihar, while three belonged to minority communities in Kashmir. the home minister was briefed about the steps taken to eliminate terrorism from Jammu and Kashmir and counter-infiltration measures. he was briefed about the steps taken to eliminate terrorism from Jammu and Kashmir.
